In [17]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
def get_links(i):
    links=[]
    r = requests.get('https://www.breitbart.com/tag/climate-change/page/{}'.format(i))
    soup=BeautifulSoup(r.text, 'html.parser')
    
    aList = soup.find('section', {'class': 'aList'})
    for a in aList.findAll('a'):
        if a.parent.name == 'article' and "clips" not in a['href']:
            links.append('https://www.breitbart.com' + a['href'])
    return links

def get_text(link):
    article_text=''
    title=''
    author=''
    date=''
    
    r=requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    title = soup.find('h1').text
    date = soup.find('time').text
    author = soup.find('address').text
    article = soup.find('div', {'class':'entry-content'})
    headers= article.findAll('h2')
    
    if headers:
        for h2 in headers:
            article_text += h2.text
    for p in article.findAll('p'):
        article_text = article_text + "\n" + p.text
    return title, author, date, article_text

In [38]:
titles = []
authors = []
dates = []
articles=[]

for i in range(1,87):
#     print('Page: {}'.format(i))
    links=get_links(i)
    for link in links:
        title, author, date, article = get_text(link)
        
        titles.append(title)    
        authors.append(author)
        dates.append(date)
        articles.append(article) 

In [48]:
len(articles)

2083

In [42]:
articles[1000]

'TEL AVIV – Palestinian President Mahmoud Abbas accused Israel at the United Nations on Friday of destroying the environment by building settlements in the West Bank.\nSpeaking during the signing ceremony of the Paris Agreement on climate change at the\xa0UN headquarters in New York, Abbas said that the “Israeli occupation is destroying the climate in Palestine, and the Israeli settlements are destroying the environment in Palestine.”\nHe then called on the representatives from 175 countries who were present at the landmark agreement to “help us in putting an end to the occupation and to settlements.”\nIsraeli Ambassador to the UN Danny Danon accused Abbas of using an unrelated occasion to “mislead the international community.”\n“Instead of spreading hatred here at the UN, President Abbas should act to stop Palestinian terror,” he said after signing the treaty.\n“This climate summit is supposed to be a demonstration of global unity for the sake of the future of our planet,” he added. “

In [28]:
# articles = filter(None, articles)
# titles = filter(None, titles)
# authors = filter(None, authors)
# dates = filter(None, dates)

In [34]:
# articles = list(articles)
# titles = list(titles)
# authors = list(authors)
# dates = list(dates)

In [49]:
len(articles)

2083

In [50]:
df = pd.DataFrame(data={'title': titles, 'author': authors, 'date': dates, 'article': articles}).drop_duplicates(keep='first')
df['denial?'] = [1]*len(df.index)

In [52]:
df.to_csv('../../data/right/data_breitbart.csv', index=False)
df.head()

,title,author,date,article,denial?
0,Study: California Eliminating Natural Gas Appl...,Sean Moran,29 Aug 2019,California Public Utilities Commission (CPUC) ...,1
1,Greta Thunberg Arrives in NYC: ‘Keep Fossil Fu...,Penny Starr,29 Aug 2019,"Greta Thunberg, the 16-year-old Swede and face...",1
2,Ocasio-Cortez: I Want to Have ‘One Less Child’...,Hannah Bleau,28 Aug 2019,Rep. Alexandria Ocasio-Cortez (D-NY) returned ...,1
3,"Andrew Yang Advocates for ‘Space Mirrors,’ Hig...",Penny Starr,27 Aug 2019,Entrepreneur and Democrat presidential hopeful...,1
4,Beto O’Rourke Grills a Cheeseburger to Prove H...,Hannah Bleau,26 Aug 2019,"Presidential hopeful Beto O’Rourke (D), still ...",1
